In [199]:
import pandas as pd
import pymongo
import requests
from bs4 import BeautifulSoup
import time
import json
from collections import Counter

In [2]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['chess_predictions']  # Use (or create) a database called 'chess_predictions'
players = db['players']  # Use (or create) a collection called 'players'

In [3]:
def get_player_profile(username):
    try:
        response = requests.get(f'https://api.chess.com/pub/player/{username}')
        return json.loads(response.content.decode('utf-8'))
    except:
        return [] 
        
def get_player_stats(username):
    try:
        response = requests.get(f'https://api.chess.com/pub/player/{username}/stats')
        return json.loads(response.content.decode('utf-8'))
    except:
        return []
        
def get_player_games(username):
    '''return a list of lists where each list contains the games played each the months of January through May 2018'''
    
    months = ['01', '02', '03', '04', '05']
    month_games = []
    for month in months:
        try:
            response = requests.get(f'https://api.chess.com/pub/player/{username}/games/2018/{month}')
            month_games.append([json.loads(response.content.decode('utf-8'))])
        except:
            continue
    return month_games

def player_data_to_mongoDB(username, mongoDB_connection, database, collection):
    
    mc = mongoDB_connection
    
    #use/create a database
    db = mc[database]
    
    #use/create a collection
    collection = db[collection]
    
    #query Chess.com api for data
    profile = get_player_profile(username)
    stats = get_player_stats(username)
    games = get_player_games(username)
    
    #insert player data into database
    collection.insert_one({**profile,
                           **stats,
                           'games': games
                            })

def all_player_data_to_mongoDB(players, mongoDB_connection, database, collection, verbose=False):
    '''insert all player data into a mongoDB
       Params:
          players: list of player names
          mongoDB_connection: open connection to database ex. pymongo.MongoClient()
          database: name of database
          collection: name of collection
       Keyword Args:
          verbose: if True print player name after each insert, default is False
    '''
    for player in players:
        player_data_to_mongoDB(player, mongoDB_connection, database, collection)
        if verbose:
            print(player)

In [224]:
player_data_to_mongoDB('babu500',pymongo.MongoClient(), 'chess_predictions', 'players')

In [18]:
df = pd.DataFrame(list(db['players'].find()))
df.head()

,@id,_id,avatar,chess960_daily,chess_blitz,chess_bullet,chess_daily,chess_rapid,country,followers,games,joined,last_online,location,name,player_id,status,url,username
0,https://api.chess.com/pub/player/babu500,5afa039f098388ad5ff6a3a1,NaN,NaN,"{'last': {'rating': 777, 'date': 1525820389, '...",NaN,"{'last': {'rating': 1200, 'date': 1516320601, ...","{'last': {'rating': 974, 'date': 1522794278, '...",https://api.chess.com/pub/country/US,1,[[{'games': [{'url': 'https://www.chess.com/li...,1516059527,1526310229,NaN,NaN,42361082,premium,https://www.chess.com/member/babu500,babu500
1,https://api.chess.com/pub/player/0rlandomagic,5afa05ce098388c83c3f08fa,NaN,"{'last': {'rating': 1257, 'date': 1526259679, ...","{'last': {'rating': 1800, 'date': 1526089345, ...","{'last': {'rating': 2117, 'date': 1526323151, ...","{'last': {'rating': 1425, 'date': 1525869836, ...",NaN,https://api.chess.com/pub/country/US,13,[[{'games': [{'url': 'https://www.chess.com/da...,1515424234,1526322679,NaN,NaN,42121838,basic,https://www.chess.com/member/0rlandoMagic,0rlandomagic
2,https://api.chess.com/pub/player/0ldtower,5afa063e098388c83c3f08fc,NaN,NaN,NaN,NaN,NaN,NaN,https://api.chess.com/pub/country/US,0,"[[{'games': []}], [{'games': []}], [{'games': ...",1515958780,1526117639,NaN,Nick Chamberlain,42324950,basic,https://www.chess.com/member/0ldTower,0ldtower
3,https://api.chess.com/pub/player/0rlandomagic,5afa0640098388c83c3f08fd,NaN,"{'last': {'rating': 1257, 'date': 1526259679, ...","{'last': {'rating': 1800, 'date': 1526089345, ...","{'last': {'rating': 2117, 'date': 1526323151, ...","{'last': {'rating': 1425, 'date': 1525869836, ...",NaN,https://api.chess.com/pub/country/US,13,[[{'games': [{'url': 'https://www.chess.com/da...,1515424234,1526322679,NaN,NaN,42121838,basic,https://www.chess.com/member/0rlandoMagic,0rlandomagic
4,https://api.chess.com/pub/player/0xvyper,5afa0641098388c83c3f08fe,NaN,NaN,"{'last': {'rating': 705, 'date': 1526217305, '...",NaN,NaN,"{'last': {'rating': 870, 'date': 1516238541, '...",https://api.chess.com/pub/country/US,1,[[{'games': [{'url': 'https://www.chess.com/li...,1515610811,1526216631,NaN,Vyper Dev,42192576,basic,https://www.chess.com/member/0xVyper,0xvyper


In [38]:
df_copy.drop_duplicates(subset='player_id', inplace=True)
df.reset_index(drop=True, inplace=True)

In [227]:
df.tail()

,@id,_id,avatar,chess960_daily,chess_blitz,chess_bullet,chess_daily,chess_rapid,country,followers,...,resigned,checkmated,timeout,abandoned,repetition,agreed,stalemate,timevsinsufficient,insufficient,50move
1447,https://api.chess.com/pub/player/onubani,5afa0ddd098388c83c3f0ef1,https://images.chesscomfiles.com/uploads/v1/us...,"{'last': {'rating': 1200, 'date': 1517894420, ...","{'last': {'rating': 930, 'date': 1521988328, '...","{'last': {'rating': 781, 'date': 1517547924, '...","{'last': {'rating': 1074, 'date': 1518857403, ...","{'last': {'rating': 1109, 'date': 1526208290, ...",https://api.chess.com/pub/country/US,0,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1448,https://api.chess.com/pub/player/jrdele,5afa0de4098388c83c3f0ef3,NaN,NaN,"{'last': {'rating': 1709, 'date': 1526336980, ...",NaN,NaN,NaN,https://api.chess.com/pub/country/US,0,...,22.0,11.0,26.0,NaN,2.0,NaN,NaN,1.0,NaN,NaN
1449,https://api.chess.com/pub/player/ed-ky,5afa0de5098388c83c3f0ef4,NaN,NaN,"{'last': {'rating': 541, 'date': 1517671309, '...",NaN,"{'last': {'rating': 862, 'date': 1525812799, '...",NaN,https://api.chess.com/pub/country/US,1,...,2.0,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1450,https://api.chess.com/pub/player/kiddwerks,5afa0de7098388c83c3f0ef5,https://images.chesscomfiles.com/uploads/v1/us...,NaN,"{'last': {'rating': 553, 'date': 1525747639, '...","{'last': {'rating': 1080, 'date': 1521786794, ...",NaN,NaN,https://api.chess.com/pub/country/US,0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451,https://api.chess.com/pub/player/jojomcfee1,5afa0de8098388c83c3f0ef6,NaN,NaN,"{'last': {'rating': 744, 'date': 1525741886, '...",NaN,"{'last': {'rating': 1133, 'date': 1526078549, ...","{'last': {'rating': 1004, 'date': 1525708090, ...",https://api.chess.com/pub/country/US,2,...,NaN,6.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [226]:
games_column = df['games']
player_idx = range(df.shape[0])
for player in player_idx:
#     print(player)
    rated = []
    time_class = []
    rules = []
    eco = []
    results = []
    if len(games_column[player]) != 5:
        continue
    else:
        for month in range(4):
            try:
                for game in range(games_in_a_month(games_column, player, month)):
                    try:
                        rated.append(rated_games(games_column, player, month, game))
                    except KeyError:
                        continue
                    try:
                        time_class.append(time_class_games(games_column, player, month, game))
                    except KeyError:
                        continue
                    try:
                        rules.append(rules_games(games_column, player, month, game))
                    except KeyError:
                        continue
                    try:
                        eco.append(eco_games(games_column, player, month, game))
                    except KeyError:
                        continue
                    try:
                        results.append(results_games(games_column, player, month, game))
                    except KeyError:
                        continue
            except KeyError:
                continue

        make_columns(time_class, df, player)
        make_columns(rated, df, player)
        make_columns(rules, df, player)
        make_columns(eco, df, player)
        make_columns(results, df, player)


In [225]:
def games_in_a_month(games_column, player, month):
    return len(games_column[player][month][0]['games'])
    
def rated_games(games_column, player, month, game):
    return str(games_column[player][month][0]['games'][game]['rated'])

def time_class_games(games_column, player, month, game):
    return games_column[player][month][0]['games'][game]['time_class']

def rules_games(games_column, player, month, game):
    return games_column[player][month][0]['games'][game]['rules']

def eco_games(games_column, player, month, game):
    return games_column[player][month][0]['games'][game]['eco'][31:].split('-')[0]

def results_games(games_column, player, month, game):
    if games_column[player][month][0]['games'][game]['white'] == df.username[0]:
        return games_column[player][month][0]['games'][game]['white']['result']
    else:
        return games_column[player][month][0]['games'][game]['black']['result']

def make_columns(features, df, index):
        counters = Counter()
        for feature in features:
            counters[feature] += 1

        for counter in counters:
            df.loc[index, counter] = counters[counter]


In [135]:
df.drop(labels=['standard', 'daily', 'blitz', 'lightning'],axis=1, inplace=True)

In [137]:
df.drop(labels=['True', 'False'],axis=1, inplace=True)

In [ ]:
df['games'][player][month][0]['games'][one game from the month]['url',
                                                                'pgn',
                                                                'time_control',
                                                                'end_time',
                                                                'rated',
                                                                'fen',
                                                                'time_class',
                                                                'rules',
                                                                'white',
                                                                'black']


In [182]:
eco = []
for game in range(len(games[1][0][0]['games'])):
    try:
        eco.append(games[0][0][0]['games'][game]['eco'][31:].split('-')[0])
    except:
        continue
print(eco)

['C50', 'C24', 'C48', 'C47', 'C25', 'C47', 'C48', 'C46', 'C50']


In [206]:
games[0][0][0]['games'][0]['pgn'].split('} ')[-1]

'0-1'

In [213]:
def results():
    if games[0][0][0]['games'][0]['white'] == df.username[0]:
        return games[0][0][0]['games'][0]['white']['result']
    else:
        return games[0][0][0]['games'][0]['black']['result']

{'@id': 'https://api.chess.com/pub/player/babu500',
 'rating': 953,
 'result': 'checkmated',
 'username': 'babu500'}

In [214]:
df.username[0]

'babu500'

In [215]:
games[0][0][0]['games'][0]['black']['result']

'win'

In [228]:
games[0][0][0]['games'][0]['white']

{'@id': 'https://api.chess.com/pub/player/babu500',
 'rating': 953,
 'result': 'checkmated',
 'username': 'babu500'}

In [232]:
letters = ['A', 'B', 'C', 'D', 'E']
nums = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
eco_codes = []
for letter in letters:
    for num in nums:
        for num2 in nums:
            eco_codes.append(letter + num + num2)
print(eco_codes)

['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65', 'A66', 'A67', 'A68', 'A69', 'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78', 'A79', 'A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A90', 'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97', 'A98', 'A99', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'B31', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39', 'B40', 'B41', 'B42'